In [19]:
#cleaning excel file
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait,Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.support import expected_conditions as EC
import time

df = pd.read_excel(r'C:\Users\ramchander\Documents\Data Catalog\List of SAP reports.xlsx')

df['Tags'] = df[df.columns[6:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
df.rename(columns={'Hyperlink in PROD':'Hyperlinks'}, inplace=True) #to rename a column header
df.drop(df.columns[6:27],inplace=True,axis=1)
df.drop(df.columns[3:5],inplace=True,axis=1)
df.drop('Parent Folder Path',inplace=True,axis=1) #to remove a column
df['Name'] =df['Name'].map(lambda x: x.strip()) #to strip off space at end of column
final = df[0:6] #to reduce the row count to 5 for test 
final.to_excel('Test6.xlsx')

In [1]:
#code to upload the report

test = pd.read_excel(r'C:\Users\ramchander\Documents\Data Catalog\test5.xlsx')
#test.fillna('',inplace=True,axis=1)

#test.Name = test.Name.str.rstrip()
text_file = open("Report.txt", "w").close()

def logf(message,output):
    text_file = open("Report.txt", "a")
    joinedmsg = ' '.join([message,output])
    text_file.write(joinedmsg)
    text_file.write("\n")
    text_file.close()

request_access = 'Request access to this universe using the process at https://documents.health.usf.edu/pages/viewpage.action?pageId=418021758'


binary = FirefoxBinary('C:\\Program Files (x86)\\Mozilla Firefox\\firefox.exe')

driver = webdriver.Firefox(firefox_binary=binary,executable_path="C:\\Users\\ramchander\\Downloads\\geckodriver-v0.20.1-win64\\geckodriver.exe")

driver.get("https://use-su1.azuredatacatalog.com/#/publish")
driver.implicitly_wait(2)

x,_ = test.shape


for i in range(0,5):
    try:
        driver.get("https://use-su1.azuredatacatalog.com/#/publish")

        #wait for the publish page to load               
        WebDriverWait(driver, 100).until(EC.visibility_of_element_located((By.ID,'manual-entry-button')))
        driver.find_element_by_id("manual-entry-button").click()
        #wait for the manual entry tab to load
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME,'manual-entry-input')))
        #copy the report name
        driver.find_element_by_css_selector('.source-name > input:nth-child(2)').send_keys(test.Name[i])

        #print('Name done',i)

        driver.find_element_by_css_selector('.request-access > textarea:nth-child(2)').send_keys(request_access)
        #print('Request access done')

        #Select http as the source type
        source_type = Select(driver.find_element_by_css_selector('.source-type > select:nth-child(2)'))
        source_type.select_by_visible_text('Http')

        #select report as the object type
        object_type = Select(driver.find_element_by_css_selector('.object-type > select:nth-child(2)'))
        object_type.select_by_visible_text('report')

                #copy the link to the report
        driver.find_element_by_css_selector('#properties\.dsl\.address\.url > input:nth-child(2)').send_keys(test.Hyperlinks[i])

       # print('Type & URL done')     

        experts =  driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[2]/div/div/div[10]/div[2]/div/div[2]/div[1]')
        experts.click()
        experts.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[2]/div/div/div[10]/div[2]/div/div[2]/div[2]/span/input').send_keys(test.Experts[i])
        expertss = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[2]/div/div/div[10]/div[2]/div/div[2]/div[2]/img')
        expertss.click()
        #print('Experts done')

        tags =  driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[2]/div/div/div[11]/div/div/div[2]/div[1]')
        tags.click()
        tags.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[2]/div/div/div[11]/div/div/div[2]/div[2]/span/input').send_keys(test.Tags[i])
        tagss = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[2]/div[2]/div/div/div[11]/div/div/div[2]/div[2]/img')
        tagss.click()
        #print('Tags done')

        time.sleep(2)
        driver.find_element_by_css_selector('button.btn:nth-child(2)').click()
        #wait for the sortby to load
        #print('Portal transition done')
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR,'div.search-group:nth-child(5) > div:nth-child(2) > a:nth-child(1)')))
        driver.find_element_by_css_selector('div.search-group:nth-child(5) > div:nth-child(2) > a:nth-child(1)').click()
        driver.find_element_by_css_selector('ul.dropdown-menu:nth-child(3) > li:nth-child(3) > a:nth-child(1)').click()
        #print('Sort done')
        time.sleep(3)
        #wait for relevant tile to load
        WebDriverWait(driver, 20).until(EC.text_to_be_present_in_element((By.CSS_SELECTOR,'div.tile:nth-child(3) > div:nth-child(3) > div:nth-child(1)'),test.Name[i][:20]))
        driver.find_element_by_css_selector('div.tile:nth-child(3) > div:nth-child(3) > div:nth-child(1)').click()
        #print('Relevant Tile load done')

        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME,'take-ownership')))
        driver.find_element_by_class_name('take-ownership').click()
        #print('Ownership taken')

        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME,'visibility-toggle')))
        driver.find_element_by_css_selector('.visibility-toggle > div:nth-child(2)').click()
        #print('Visibility switched')


        permissions = driver.find_element_by_class_name('users-control').find_element_by_class_name('add-btn')
        permissions.click()
        permissions.find_element_by_xpath('//input[@placeholder="add users and groups separated by commas"]').send_keys(test.contributer[i])
        permissions.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div[2]/div/div/div[4]/div[1]/div[2]/div[1]/div/div/div/div[6]/div/div[3]/div[3]/div[5]/div/div[2]/div[2]/img').click()
        #print('Read/write done')


        permissions = driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div[2]/div/div/div[4]/div[1]/div[2]/div[1]/div/div/div/div[6]/div/div[3]/div[3]/div[7]/div/div[2]/div[1]')
        permissions.click()
        permissions.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div[2]/div/div/div[4]/div[1]/div[2]/div[1]/div/div/div/div[6]/div/div[3]/div[3]/div[7]/div/div[2]/div[2]/span/input').send_keys(test.consumer[i])
        permissions.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div[2]/div/div/div[4]/div[1]/div[2]/div[1]/div/div/div/div[6]/div/div[3]/div[3]/div[7]/div/div[2]/div[2]/img').click()
        #print('Read only Done')

        permissions = driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div[2]/div/div/div[4]/div[1]/div[2]/div[1]/div/div/div/div[6]/div/div[3]/div[3]/div[9]/div/div[2]/div[1]')
        permissions.click()
        permissions.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div[2]/div/div/div[4]/div[1]/div[2]/div[1]/div/div/div/div[6]/div/div[3]/div[3]/div[9]/div/div[2]/div[2]/span/input').send_keys(test.no_access[i])
        permissions.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div[2]/div/div/div[4]/div[1]/div[2]/div[1]/div/div/div/div[6]/div/div[3]/div[3]/div[9]/div/div[2]/div[2]/img').click()
        #print('No access done')
        time.sleep(3)
        driver.implicitly_wait(10)
        #driver.find_element_by_css_selector('#main-navigation > div:nth-child(2) > a:nth-child(1)').click()
        driver.get("https://use-su1.azuredatacatalog.com/#/home")
        #logf(str(i))
        logf("Done -",str(i))
        print("done")
    except:
        print("Exception")
        logf("Error in Row -",str(i))
       # driver.get('https://use-su1.azuredatacatalog.com/#/browse')

done
done
done
Exception
done


In [1]:
import selenium
help(selenium)

Help on package selenium:

NAME
    selenium

FILE
    c:\users\ramchander\appdata\local\continuum\anaconda2\lib\site-packages\selenium\__init__.py

DESCRIPTION
    # Licensed to the Software Freedom Conservancy (SFC) under one
    # or more contributor license agreements.  See the NOTICE file
    # distributed with this work for additional information
    # regarding copyright ownership.  The SFC licenses this file
    # to you under the Apache License, Version 2.0 (the
    # "License"); you may not use this file except in compliance
    # with the License.  You may obtain a copy of the License at
    #
    #   http://www.apache.org/licenses/LICENSE-2.0
    #
    # Unless required by applicable law or agreed to in writing,
    # software distributed under the License is distributed on an
    # "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
    # KIND, either express or implied.  See the License for the
    # specific language governing permissions and limitations
    # under t